In [2]:
!pip install ortools
from ortools.sat.python import cp_model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [28]:
model=cp_model.CpModel()
ub=50
x=model.new_int_var(0,ub,"x")
y=model.new_int_var(0,ub,"y")
z=model.new_int_var(0,ub,"z")
model.add(2 * x + 7 * y + 3 * z <= 50)
model.add(3 * x - 5 * y + 7 * z <= 45)
model.add(5 * x + 2 * y - 6 * z <= 37)

model.maximize(2 * x + 2 * y + 3 * z)
solver=cp_model.CpSolver()
status=solver.Solve(model)
if status==  cp_model.OPTIMAL or status==cp_model.FEASIBLE:
  print(solver.objective_value)
  print(f"x={solver.value(x)}")
  print(f"y={solver.Value(y)}")
  print(f"z={solver.Value(z)}")
else:
  print("None")

print(f"  status   : {solver.status_name(status)}")
print(f"  conflicts: {solver.num_conflicts}")
print(f"  branches : {solver.num_branches}")
print(f"  wall time: {solver.wall_time} s")

class AllSolutions(cp_model.CpSolverSolutionCallback):
  def __init__(self,variables :list[cp_model.IntVar]):
    cp_model.CpSolverSolutionCallback.__init__(self)
    self.__variables = variables
    self.__solution_count = 0

  def on_solution_callback(self) -> None:
        self.__solution_count += 1
        for v in self.__variables:
            print(f"{v}={self.value(v)}  ")
        print()

  @property
  def solution_count(self) -> int:
    return self.__solution_count
solver=cp_model.CpSolver()
solution_printer=AllSolutions([x,y,z])
solver.parameters
status=solver.solve(model,solution_printer)

35.0
x=7
y=3
z=5
  status   : OPTIMAL
  conflicts: 2
  branches : 12
  wall time: 0.00816447 s
x=0  
y=0  
z=0  

x=0  
y=7  
z=0  

x=5  
y=5  
z=0  

x=9  
y=3  
z=3  

x=9  
y=2  
z=4  

x=7  
y=3  
z=5  



In [29]:
from ortools.sat.python import cp_model

# Product and Slot Info
products = [
    {"id": 1, "frequency": 15, "volume": 2},
    {"id": 2, "frequency": 8,  "volume": 1},
    {"id": 3, "frequency": 20, "volume": 3}
]

slots = [
    {"id": 1, "distance": 1, "capacity": 3},
    {"id": 2, "distance": 2, "capacity": 2},
    {"id": 3, "distance": 3, "capacity": 2}
]

model = cp_model.CpModel()
assignment = {}

# Step 1: Create Boolean variables
for i in range(len(products)):
    for j in range(len(slots)):
        assignment[(i, j)] = model.new_bool_var(f"p{products[i]['id']}_s{slots[j]['id']}")

# Step 2: Each product must go in exactly one slot
for i in range(len(products)):
    model.add(sum(assignment[(i, j)] for j in range(len(slots))) == 1)

# Step 3: Each slot's capacity must not be exceeded
for j in range(len(slots)):
    model.add(
        sum(assignment[(i, j)] * products[i]["volume"] for i in range(len(products)))
        <= slots[j]["capacity"]
    )

# Step 4: Objective - Minimize total walking distance
model.Minimize(
    sum(
        assignment[(i, j)] * products[i]["frequency"] * slots[j]["distance"]
        for i in range(len(products))
        for j in range(len(slots))
    )
)

# Step 5: Solve
solver = cp_model.CpSolver()
status = solver.solve(model)

# Step 6: Display Result
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    for i in range(len(products)):
        for j in range(len(slots)):
            if solver.value(assignment[(i, j)]) == 1:
                print(f"Product {products[i]['id']} assigned to Slot {slots[j]['id']}")
    print(f"Minimum Total Walking Distance: {solver.objective_value}")
else:
    print("No solution found.")


Product 1 assigned to Slot 2
Product 2 assigned to Slot 3
Product 3 assigned to Slot 1
Minimum Total Walking Distance: 74.0
